## es para generar un clasificador como baseline. El problema es que en ingles utiliza la pregunta
## se debe entrenar para español sin utilizar la pregunta, solo doc + claim

In [58]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [30]:
!pip install transformers

You should consider upgrading via the 'C:\Users\Edu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [31]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

You should consider upgrading via the 'C:\Users\Edu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [32]:
notebook_login()



In [33]:
from transformers import AutoTokenizer,AutoModel, RobertaTokenizer

# Load the BERT tokenizer.
print('Loading roBERT tokenizer...')

tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

Loading roBERT tokenizer...


In [59]:
import json #English/test_set_english_claim.json
with open("../../data/Spanish/train_spanish_claims.json", encoding='utf-8') as f:
  data = json.load(f)
len(data)

1506

In [60]:
#data[0]

In [61]:
# split test_data 
# Calculate split index
split_index = int(0.8 * len(data))

# Split the data
train_data = data[:split_index]
val_data = data[split_index:]
print(f"Train size: {len(train_data)}")
print(f"Test size: {len(val_data)}")

Train size: 1204
Test size: 302


In [62]:
train_data[0]

{'claim': 'Correos te pide 1,4€ por desinfectarte los paquetes',
 'crawled_date': '2020-04-25',
 'country_of_origin': 'spain',
 'doc': ' | 6 min lectura Nos habéis preguntado por otro mensaje relacionado con la marca española de automóviles SEAT y respiradores. En este habla un supuesto trabajador en nombre del empresa diciendo que la automovilística está esperando a que los hospitales españoles les pidan respiradores, y ofrecería un manual de instrucciones de los mismos. Adjunta además una dirección de correo electrónico y un teléfono. Este mensaje es falso. Así lo ha desmentido la compañía a Newtral.es. Ni han lanzado ninguna campaña similar, ni están teniendo problemas de distribución de los aparatos sanitarios. Una cadena de WhatsApp supuestamente proveniente de la Guardia Civil recomienda desinstalar la aplicación de videoconferencia Zoom, además de eliminar la cuenta. La cadena adjunta un tuit enviado por la cuenta oficial de la Guardia Civil. Sin embargo el tuit no dice eso, por

In [63]:
val_claims = []
for fact in val_data:
        strencode = fact["claim"].encode("ascii", "ignore")
        strdecode = strencode.decode()
        val_claims.append(strdecode.lower().strip())

train_claims = []
for fact in train_data:
        strencode = fact["claim"].encode("ascii", "ignore")
        strdecode = strencode.decode()
        train_claims.append(strdecode.lower().strip())


In [39]:
len(val_claims)

377

In [40]:
len(train_claims)

1506

In [41]:
# import json
# with open("../../data/English/train_claims_quantemp.json") as f:
#   train_data = json.load(f)
# len(train_data)

In [42]:
# train_claims = []
# for fact in train_data:
#         strencode = fact["claim"].encode("ascii", "ignore")
#         strdecode = strencode.decode()
#         train_claims.append(strdecode.lower().strip())
# '''
# for claim in train_claims:
#   if claim in test_claims:
#     print(claim)'''

In [43]:
# import json
# with open("../../data/English/decomposed_questions_with_mapped_bm25_evidence/train_claimdecomp_evidence_question_mapping.json") as f:
#   train_data = json.load(f)
# len(train_data
#     )

In [44]:
# import json
# with open("../../data/English/decomposed_questions_with_mapped_bm25_evidence/val_claimdecomp_evidence_question_mapping.json") as f:
#   val_data = json.load(f)
# len(val_data
#     )

In [45]:

#val_data[2]

In [64]:

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [66]:
def get_features(data2):
  features = []
  for index, fact in enumerate(data2):
    claim = fact["claim"]
    evidences = []
    #for doc in fact["docs"]:
      #if len(question["top_k_doc"])>0:
    #evidences.append(doc)
    evidences.append(fact["doc"])
      #questions.append(question["questions"])
    #questions = list(set(questions))
    evidences = list(set(evidences))
    feature = "[Claim]:"+claim+"[Evidences]:"+" ".join(evidences)
    features.append(feature)
  return features




In [67]:
train_data[0]

{'claim': 'Correos te pide 1,4€ por desinfectarte los paquetes',
 'crawled_date': '2020-04-25',
 'country_of_origin': 'spain',
 'doc': ' | 6 min lectura Nos habéis preguntado por otro mensaje relacionado con la marca española de automóviles SEAT y respiradores. En este habla un supuesto trabajador en nombre del empresa diciendo que la automovilística está esperando a que los hospitales españoles les pidan respiradores, y ofrecería un manual de instrucciones de los mismos. Adjunta además una dirección de correo electrónico y un teléfono. Este mensaje es falso. Así lo ha desmentido la compañía a Newtral.es. Ni han lanzado ninguna campaña similar, ni están teniendo problemas de distribución de los aparatos sanitarios. Una cadena de WhatsApp supuestamente proveniente de la Guardia Civil recomienda desinstalar la aplicación de videoconferencia Zoom, además de eliminar la cuenta. La cadena adjunta un tuit enviado por la cuenta oficial de la Guardia Civil. Sin embargo el tuit no dice eso, por

In [68]:
train_features = get_features(train_data)

In [69]:
len(train_features)

1204

In [70]:
train_features[1]

'[Claim]:Una pareja senegalesa inventó 18 hijos y defraudó 360.000 euros en ayudas sociales en España[Evidences]:"Recordáis la pareja senegalesa que defraudó 360.000 € en ayudas, fingiendo tener 18 hijos. Pues la han condenado a pagar dos euros. Si fueran españoles, van 10 años a la cárcel". Con mensajes como este se difunde el caso de una pareja que fingió tener 18 hijos y defraudó 360.000 euros en ayudas sociales afirmando que es senegalesa y que ocurrió en España. Es un bulo: el caso ocurrió en Sunderland (Reino Unido) y la pareja no es senegalesa, sino inglesa.  Una foto publicada por medios ingleses de la pareja, Tracy Ashbridge y Robert Ashbridge de Sunderland, Inglaterra, condenados por fraude de 360.000 euros en ayudas sociales.  El caso ocurrió en Sunderland (Reino Unido) y la pareja no es senegalesa, sino inglesa  Estos contenidos se difunden con capturas de titulares en medios donde no se afirma ni la nacionalidad de los acusados ni el lugar en el que ocurrió. Pero es un cas

In [71]:
val_features = get_features(val_data)

In [72]:
len(val_features)

302

In [73]:
val_features[2]

'[Claim]:Tomado de una cámara instalada en una de las casas donde ocurrió la erupción del volcán de Isla Blanca en Nueva Zelanda el 9 de diciembre del 2019 donde 18 personas perdieron la vida[Evidences]: | 2 min lectura Está volviendo a circular por redes sociales un vídeo que supuestamente muestra la erupción del volcán Isla Blanca, en Nueva Zelanda. Las imágenes compartidas exhiben un mar tranquilo en el que de repente se produce una explosión submarina que genera una nube de humo que se expande rápidamente y acaba cegando la cámara. Sin embargo, el vídeo no es real, sino que se trata de una simulación. También nos habéis enviado a través de nuestro servicio de verificación por WhatsApp (+34 627 28 08 15) estas mismas imágenes acompañadas de un mensaje que indica que se trata de un vídeo “tomado de una cámara instalada en una de las casas donde ocurrió la erupción del volcán de Isla Blanca el 9 de diciembre del 2019, donde 18 personas perdieron la vida”. Es cierto que el 9 de diciemb

In [74]:
train_data[0].keys()

dict_keys(['claim', 'crawled_date', 'country_of_origin', 'doc', 'fact_source', 'factchecker', 'label', 'url', 'lang', 'numerical', 'taxonomy_label'])

In [75]:
train_labels = [fact["label"] for fact in train_data]
val_labels = [fact["label"] for fact in val_data]

In [76]:
train_labels_final = LE.fit_transform(train_labels)
train_labels_final

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [77]:
train_labels_final[:20]

array([1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int64)

In [78]:
val_labels_final = LE.transform(val_labels)
val_labels_final

array([1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1,
       2, 1, 1, 0, 2, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,

In [79]:
val_data[-1]

{'claim': 'ABC tuitea que cuatro de cada cinco jóvenes del colectivo LGTBI se sentirían más protegidos en un país islámico que en una España gobernada por Vox',
 'crawled_date': '2024-04-20T03:15:56',
 'country_of_origin': None,
 'doc': '"Cuatro de cada cinco jóvenes pertenecientes al colectivo LGTB reconocen que se sentirían más protegidos en un país islámico que en una España gobernada por Vox". Este es el mensaje que se está difundiendo en Twitter (ahora X) como si lo hubiera publicado ABC. Es un bulo. La cuenta que lo publica no es la de ABC, sino @Santodebot, un perfil del que ya hemos desmentido otros contenidos y que en su biografía se define como "parodia".  Este es el contenido que se difunde:  Captura del bulo que se difunde.  La cuenta que lo publica no es la de ABC  El tuit se difunde como si lo hubiera publicado el diario ABC, con mensajes como estos: "La propaganda ha calado bien hondo entonces" o "esto es producto del adoctrinamiento".  Si nos fijamos en la cuenta vemos 

In [80]:
len(val_labels_final)

302

In [81]:
input_ids = []
attention_masks = []

for sent in train_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', train_features[0])
print('Token IDs:', input_ids[0])

c:\Users\Edu\anaconda3\envs\clef\lib\site-packages\transformers\tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  [Claim]:Correos te pide 1,4€ por desinfectarte los paquetes[Evidences]: | 6 min lectura Nos habéis preguntado por otro mensaje relacionado con la marca española de automóviles SEAT y respiradores. En este habla un supuesto trabajador en nombre del empresa diciendo que la automovilística está esperando a que los hospitales españoles les pidan respiradores, y ofrecería un manual de instrucciones de los mismos. Adjunta además una dirección de correo electrónico y un teléfono. Este mensaje es falso. Así lo ha desmentido la compañía a Newtral.es. Ni han lanzado ninguna campaña similar, ni están teniendo problemas de distribución de los aparatos sanitarios. Una cadena de WhatsApp supuestamente proveniente de la Guardia Civil recomienda desinstalar la aplicación de videoconferencia Zoom, además de eliminar la cuenta. La cadena adjunta un tuit enviado por la cuenta oficial de la Guardia Civil. Sin embargo el tuit no dice eso, por tanto se trata de un mensaje falso. La pregunta a Dul

In [82]:
val_input_ids = []
val_attention_masks = []

for sent in val_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    val_input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    val_attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', val_features[0])
print('Token IDs:', val_input_ids[0])

Original:  [Claim]:Acaba de salir un comunicado oficial de pepsi que no consuman 7up porque le echaron ácido muriático por error y ya van 8 muertos[Evidences]:“Acaba de salir un comunicado oficial de pepsi que no consuman 7up porque le echaron ácido muriático por error y ya van 8 muertos (sic)”, dice una imagen que circula en Facebook y Whatsapp. Pero es falso, se trata de desinformación que circula en internet al menos de 2017. A pesar de que la imagen contiene faltas de ortografía y verificadores de otros países lo desmintieron en años anteriores, usuarios de Facebook y WhatsApp compartieron recientemente la falsa alerta La empresa ya lo desmintió El 23 de septiembre de 2017, en la página oficial de Facebook del refresco 7Up se publicó un desmentido en donde narraba que el supuesto comunicado que recomendaba no consumir sus productos no era oficial y no había sido emitido por la empresa. Aprovecharon para destacar que siguen protocolos de seguridad en sus productos y que tienen estri

In [83]:
train_labels_final = torch.tensor(train_labels_final)
val_labels_final = torch.tensor(val_labels_final)

In [84]:
val_labels_final.shape

torch.Size([302])

In [85]:
len(val_input_ids)

302

In [86]:
num_classes = len(list(set(train_labels)))
list(set(train_labels))

['True', 'False', 'Conflicting']

In [87]:

num_classes

3

In [88]:
LE.classes_

array(['Conflicting', 'False', 'True'], dtype='<U11')

In [89]:
from torch.utils.data import TensorDataset, random_split
# train_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)
# difficulty_tensor = torch.tensor(difficulty_level_vectors,dtype=torch.float)
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, train_labels_final)
val_dataset = TensorDataset(val_input_ids, val_attention_masks,val_labels_final)
#

In [90]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler(dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [91]:
from torch import nn
class MultiClassClassifier(nn.Module):
    def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
        super().__init__()

        self.roberta = AutoModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=True)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # # nn.ReLU(),
            # # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            nn.Linear(mlp_dim, labels_count)
        )
        # self.softmax = nn.LogSoftmax(dim=1)
        if freeze_bert:
            print("Freezing layers")
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, tokens, masks):
        output = self.roberta(tokens, attention_mask=masks)
        dropout_output = self.dropout(output["pooler_output"])
        # concat_output = torch.cat((dropout_output, topic_emb), dim=1)
        # concat_output = self.dropout(concat_output)
        mlp_output = self.mlp(dropout_output)
        # proba = self.sigmoid(mlp_output)
        # proba = self.softmax(mlp_output)

        return mlp_output

In [92]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Loads BertForSequenceClassification, the pretrained BERT model with a single
model = MultiClassClassifier("uf-aice-lab/math-roberta",num_classes, 1024,768,140,dropout=0.1,freeze_bert=False)

# model.load_state_dict(torch.load("model_bert_difficulty_prediction/model_weights"))

# Tell pytorch to run this model on the GPU.
model.cuda()


Some weights of RobertaModel were not initialized from the model checkpoint at uf-aice-lab/math-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MultiClassClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [ ]:
# model.load_state_dict(torch.load("model_bert_difficulty_prediction/model_weights"))


In [93]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

c:\Users\Edu\anaconda3\envs\clef\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [94]:
from transformers import get_linear_schedule_with_warmup


epochs = 20

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs



In [95]:
len(train_dataloader)

76

In [96]:
1935 * 32

61920

In [97]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [98]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [99]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [100]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [101]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [102]:
for param in model.roberta.encoder.layer[0:5].parameters():
    param.requires_grad=False

In [103]:
loss_func = nn.CrossEntropyLoss()


In [104]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
early_stopping = EarlyStopping(patience=2, verbose=True)
# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_accuracy = 0
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questimport gensim.downloader as api
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        # b_poincare = batch[2].to(device)
        # b_difficulty = batch[3].to(device)
        b_labels = batch[2].to(device)
        # skill_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        probas = model(b_input_ids,b_input_mask)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        loss = loss_func(probas, b_labels)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        # scheduler.step()
        logits = probas.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_train_accuracy += flat_accuracy(logits, label_ids)
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print(" Train Accuracy: {0:.2f}".format(avg_train_accuracy))

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)



    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)

        b_input_mask = batch[1].to(device)
        # b_poincare = batch[2].to(device)
        # b_difficulty = batch[3].to(device)
        b_labels = batch[2].to(device)
        # skill_labels = batch[3].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.

          logits = model(b_input_ids,b_input_mask)

        # Accumulate the validation loss.
        loss = loss_func(logits, b_labels)
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    early_stopping(avg_val_loss, model)
    if early_stopping.early_stop:
      print("Early stopping")
      break
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    output_dir = 'math_roberta_claimdecomp_final_continued/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)
    tokenizer.save_pretrained(output_dir)
    torch.save(model.state_dict(), os.path.join(output_dir, 'model_weights'))

    !rm -rf "/content/drive/My Drive/ecir_compnumfacts/math_roberta_claimdecomp_final_continued"
    !mv math_roberta_claimdecomp_final_continued "/content/drive/My Drive/ecir_compnumfacts/"
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



======== Epoch 1 / 20 ========
Training...


RuntimeError: CUDA error: out of memory
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
tokenizer.save_pretrained("math_roberta_claimdecomp_final_continued_1")
torch.save(model.state_dict(), os.path.join("math_roberta_claimdecomp_final_continued_1", 'model_weights'))

!rm -rf "/content/drive/My Drive/ecir_compnumfacts/math_roberta_claimdecomp_final_continued_1"
!mv math_roberta_claimdecomp_final_continued_1 "/content/drive/My Drive/ecir_compnumfacts/"

In [ ]:
LE.inverse_transform([0,1,2])

In [ ]:


from huggingface_hub import login

In [ ]:
login()

In [ ]:
LE.classes_